In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader
from constants import CHROMA_SETTINGS

In [2]:
loader = PyPDFLoader("./docs/The Holy Bible NIV.pdf")
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
texts = text_splitter.split_documents(documents)

In [4]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Ernest\.conda\envs\chatLLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
db = Chroma.from_documents(texts, embedding_function, persist_directory="./db/", client_settings=CHROMA_SETTINGS)

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from constants import CHROMA_SETTINGS

c:\Users\Ernest\.conda\envs\chatLLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
check_point = "LaMini-T5-738M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(check_point)
model = AutoModelForSeq2SeqLM.from_pretrained(check_point, device_map="auto", torch_dtype=torch.float32, offload_folder="offload")

In [4]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample = True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [5]:
def qa_llm():
    llm = llm_pipeline()
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    db = Chroma(persist_directory='db', embedding_function=embeddings,client_settings=CHROMA_SETTINGS)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    return qa

In [6]:
def process_answer(instruction):
    response = ''
    instruction = instruction
    qa = qa_llm()
    generated_text = qa(instruction)
    answer = generated_text['result']
    return answer, generated_text

In [7]:
question = "Who is the son of king david?"

In [8]:
answer, metadata = process_answer(question)

In [9]:
answer

'Solomon is the son of King David.'

In [68]:
metadata

{'query': 'what is the Generative Adversarial Networks ?',
 'result': 'Generative Adversarial Networks are a class of machine learning techniques that consist of two simultaneously trained models: one (the Generator) trained to generate fake data, and the other (the Discriminator) trained to discern the fake data from real examples.',
 'source_documents': [Document(page_content='just three years later, advances in GANs enabled computers to synthesize fake faces\nwhose quality rivals high-resolution portrait photographs. In this book, we look under\nthe hood of the algorithm that made all this possible.\n1.1 What are Generative Adversarial Networks?\nGenerative Adversarial Networks (GANs)  are a class of machine learning techniques that\nconsist of two simultaneously trained models: one (the Generator ) trained to generate', metadata={'source': './docs/gans-in-action-deep-learning-with-generative-adversarial-networks.pdf', 'page': 30}),
  Document(page_content='the hood of the algorithm